In [ ]:
import sys
# !{sys.executable} -m pip install jupyter_jsmol
# Suppress PySCF warning...
import pyscf
pyscf.__config__.B3LYP_WITH_VWN5 = False
from pathlib import Path
from pyscf.gto import Mole
from pyscf.lib import logger
# Wrapper functions to perform selection for variable and fixed active space sizes
from asf.wrapper import find_from_mol, find_from_scf, sized_space_from_mol, sized_space_from_scf

# Various utility functions...
from asf.utility import compare_active_spaces, show_mos_grid, pictures_Jmol
from pyscf import gto, mp, mcscf, mrpt, dmrgscf

# Functionality for (state-averaged) CASSCF.
from pyscf.mcscf import CASSCF, state_average_mix
from pyscf.fci.direct_spin1 import FCISolver
from pyscf.fci.addons import fix_spin


In [ ]:
# ethene_geometry = """
#     C        0.6695      0.0000      0.0000
#     C       -0.6695      0.0000      0.0000
#     H        1.2320      0.0000     -0.9289
#     H        1.2320      0.0000      0.9289
#     H       -1.2320      0.0000      0.9289
#     H       -1.2320      0.0000     -0.9289
# """

iron_geometry = """
O      2.09945   -0.20712   -0.06955
H      2.65808   -0.11922   -0.85974
H      2.66428   -0.52573    0.65202
O      0.04751    0.32344   -1.70791
O     -2.09702    0.08054   -0.09941
N      0.13864    2.10970    0.23028
O     -0.16040   -2.04758   -0.25208
O     -0.05207   -0.35287    1.90361
H     -2.69555   -0.01745    0.65788
H     -2.63984    0.28975   -0.87742
H     -0.14720    2.62871   -0.60801
H      1.10480    2.39390    0.42897
H     -0.98651   -2.36883   -0.65341
H      0.54518   -2.50373   -0.74261
H     -0.11214   -1.27936    2.18843
H      0.12164    0.17676    2.69663
Fe     0.01013    0.06693   -0.14210
H     -0.44640    2.44594    1.00267

"""
# mol = Mole(atom=iron_geometry, basis="anorcc", charge=2, spin=2, verbose=logger.NOTE).build()
mol = Mole(atom=iron_geometry, basis="anorcc", charge=2, spin=2, verbose=logger.INFO).build()


In [ ]:
logger.INFO

In [ ]:
# Determine an active space with four orbitals.
active_space = sized_space_from_mol(mol, size=(18,12))
# Find one active space.
# active_space = find_from_mol(mol)

In [ ]:
print(active_space)

In [ ]:
active_space.norb

In [ ]:
# pictures_Jmol(
#     mol=mol, mo_coeff=active_space.mo_coeff, mo_list=active_space.mo_list, mo_index_title=" "
# )
# mo_images = sorted(dd for dd in Path.cwd().glob("mo_*.png"))
# show_mos_grid(mo_images, mo_list=active_space.mo_list, columns=3, figsize=(10, 10))
# # clean up files
# for filename in mo_images:
#     Path.unlink(filename)

In [ ]:
mc = CASSCF(mol, ncas=active_space.norb, nelecas=active_space.nel)
# Before carrying out the CASSCF calculation, the orbital coefficient matrix columns need to be
# ordered as core - active - virtual from left to right.
# Take care - by default, sort_mo assumes that the MO list starts indexing from one, not from zero.
mo_guess = mc.sort_mo(active_space.mo_list, active_space.mo_coeff, base=0)
_ = mc.kernel(mo_coeff=mo_guess)

In [ ]:
compare_active_spaces(mc, active_space.mo_coeff, active_space.mo_list)